In [ ]:
import json
import os

import pandas as pd
from dotenv import load_dotenv

from src.api_utils import get_statement, process_statement_data, send_request
from src.transformations import transform,

pd.options.mode.copy_on_write = True

In [ ]:
load_dotenv()
token = os.getenv("TOKEN")
if token is None:
    raise ValueError("Please set the TOKEN environment variable.")
query_id = os.getenv("QUERY_ID")
if query_id is None:
    raise ValueError("Please set the QUERY_ID environment variable.")
flex_version = os.getenv("FLEX_VERSION")
if flex_version is None:
    raise ValueError("Please set the FLEX_VERSION environment variable.")
headers_str = os.getenv("HEADERS")
if headers_str is None:
    raise ValueError("Please set the HEADERS environment variable.")
headers = json.loads(headers_str)

### Get Statement from IB via API

In [ ]:
# API Calls
reference_code = send_request(token, query_id, flex_version, headers)
df = get_statement(token, reference_code, flex_version, headers)

# Process Data
df = process_statement_data(df)

In [ ]:
df_transformed = transform(df)

In [ ]:
pd.set_option("display.max_rows", None)
df_transformed.head(10)

In [ ]:
trade_pnl_sum = df_transformed.groupby("opendateTime")["PnLRealized"].sum()
wins = (trade_pnl_sum > 0).sum()
total_trades = len(trade_pnl_sum)
win_rate = (wins / total_trades * 100) if total_trades > 0 else 0
print(wins, total_trades, win_rate)

In [ ]:
trade_pnl_sum.head(10)

In [ ]:
trade_pnl_sum = filtered_df_transformed.groupby("opendateTime")["PnLRealized"].sum()
wins = (trade_pnl_sum > 0).sum()
total_trades = len(trade_pnl_sum)
win_rate = (wins / total_trades * 100) if total_trades > 0 else 0
print(wins, total_trades, win_rate)

In [ ]:
trade_pnl_sum.head(20)

In [ ]:
wins = (filtered_df_transformed["PnLRealized"] > 0).sum()
total_trades = len(filtered_df_transformed)
win_rate = (wins / total_trades * 100) if total_trades > 0 else 0
print(wins)
print(total_trades)
print(win_rate)

In [ ]:
sum_cost = abs(filtered_df_transformed["cost"].sum()).round(2)
sum_realized = filtered_df_transformed["PnLRealized"].sum().round(2)
pcr = (sum_realized / sum_cost * 100).round(2) if sum_cost != 0 else 0
print(sum_cost)
print(sum_realized)
print(pcr)